In [47]:
import pandas as pd
import os, time

# determine 涨停
def isZhangting(s_df, index=-1):
    if (s_df.iloc[index].close/s_df.iloc[index-1].close) >= 1.099:
        return True
    else:
        return False

#determine 上涨
def isShangzhang(s_df, index=-1):
    if len(s_df.index) >= (index)+1:
        return ((s_df.iloc[index].close/s_df.iloc[index-1].close-1) > 0.02)
    else:
        return False

#determine 条件一
def isQualifiedCondition1(s_df, index=-1):
    if len(stock_df.index)>=(abs(index)+2):
        if (s_df.iloc[index].close * s_df.iloc[index].volume > 200000):
            return  (isZhangting(s_df, index-1) and isShangzhang(s_df,index))
    return False

#determine 条件二
def isQualifiedCondition2(s_df, index=-1):
    if len(stock_df.index)>=(abs(index)+3):
        if (s_df.iloc[index].close / s_df.iloc[index-2].close < 0.92) \
                and (s_df.iloc[index].close<s_df.iloc[index-1].close) \
                and (s_df.iloc[index-1].close<s_df.iloc[index-2].close)\
                and isZhangting(s_df, index-2):
            return True
    return False

#determine 条件三
def isQualifiedCondition3(s_df, index=-1):
    if len(stock_df.index)>=(abs(index)+3): 
        if isQualifiedCondition1(s_df, index-1): 
            if (s_df.iloc[index].close / s_df.iloc[index-1].close < 0.95):
                return True
        return False
        
def completeCode(code_str):
    if len(code_str)<6:
        return "0"*(6-len(code_str))+code_str
    else:
        return code_str

print (time.strftime("%M:%S"))
    
########################################
datafolder_dir = os.getcwd()+"/current"
########################################

file_list = os.listdir(datafolder_dir)

file_index = 0
select_list = []
file_dict = {}

index_post = -1
index_trading = -2

index_offset = index_trading

for file in file_list:
    #print ("Working on file %s: %s"%(file_index, file))
    file_index = file_index + 1
    if ("day" in file):
        stock_df = pd.read_csv(datafolder_dir+"/"+file)
        if isQualifiedCondition1(stock_df, index_offset):
            select_list.append("condition 1:" + completeCode(str(stock_df.iloc[0].code)))
            file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file
        if isQualifiedCondition2(stock_df, index_offset):
            select_list.append("condition 2:" + completeCode(str(stock_df.iloc[0].code)))
            file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file
        if isQualifiedCondition3(stock_df, index_offset):
            select_list.append("condition 3:" + completeCode(str(stock_df.iloc[0].code)))
            file_dict[completeCode(str(stock_df.iloc[0].code))] = datafolder_dir+"/"+file

select_list.sort()

for file in select_list:
    print (file)

print (file_dict)
print (time.strftime("%M:%S"))

for index in range(len(select_list)):
    select_list[index] = select_list[index][-6:]

print (select_list)

stock_df_list = []
for item in select_list:
    print (item)
    stock_df_list.append(pd.read_csv(file_dict[item]))

print (len(stock_df_list))
print (stock_df_list[1].tail())


20:19
condition 1:000823
condition 1:002192
condition 1:002194
condition 1:002264
condition 1:002401
condition 1:002902
condition 1:002906
condition 1:300275
condition 1:300354
condition 1:300466
condition 1:300654
condition 1:300705
condition 1:300707
condition 1:600119
condition 1:600278
condition 1:600358
condition 1:600648
condition 1:600650
condition 1:600676
condition 1:600822
condition 1:601086
condition 1:603103
condition 1:603128
condition 1:603595
condition 1:603648
condition 2:002901
condition 2:300536
condition 3:603363
{'300275': '/root/notebooks/current/day_300275.csv', '300354': '/root/notebooks/current/day_300354.csv', '002401': '/root/notebooks/current/day_002401.csv', '600119': '/root/notebooks/current/day_600119.csv', '600650': '/root/notebooks/current/day_600650.csv', '600278': '/root/notebooks/current/day_600278.csv', '600676': '/root/notebooks/current/day_600676.csv', '000823': '/root/notebooks/current/day_000823.csv', '600648': '/root/notebooks/current/day_600648

In [48]:

import tushare as ts
for index in range(len(select_list)):
    df = ts.get_realtime_quotes(select_list[index])
    # print (float(stock_df_list[index].tail(1).close), float(df.iloc[0]["price"]))
    ratio = (float(df.iloc[0]["price"]) / float(stock_df_list[index].iloc[-2].close))
    if ratio < 0.97 and ratio > 0.87:
        print (select_list[index], "diff is %s"%((1-ratio)*100))

002906 diff is 7.071285427998852
300654 diff is 10.002519526329046
600119 diff is 6.035805626598467
600278 diff is 6.182065217391308
600648 diff is 5.23715415019762
600650 diff is 6.5874730021598165
600676 diff is 7.885714285714284
601086 diff is 7.840122982321285
002901 diff is 7.6230228471001915
300536 diff is 6.938775510204087
603363 diff is 7.770270270270263


In [26]:
watch_list = ["000547", "603859", "300496", "300134", "300313", "000912", "300520", "300548", "300225", "002378", "603598", "300312", "603676", "300334", "300591", "300631", "603505", "300550", "300380", "300340", "300412", "300649", "300675", "300020", "300552", "300635", "600251", "600460", "002315", "300556", "002432", "603888", "000791", "603233", "002076", "002900", "002017", "300701", "603822", "002843", "603136", "300565", "603227", "603918"]

In [30]:
# clean up the index with numbers, and make date the default index
def pre_process(df):
    df.set_index('date', inplace=True)
    return df

def obtain_daily_data(sk_code, target_dir = ""):
    if target_dir!="":
        target_dir=target_dir+os.path.sep

    day_df = pre_process(ts.get_k_data(sk_code))
    return day_df

def get_price(sk_code):
    df = obtain_daily_data(sk_code)
    return df.iloc[-2]["close"]



In [40]:
last_price_map = {}
current_price_map = {}
for stk_code in watch_list:
    last_price_map[stk_code] = get_price(stk_code)
    df = ts.get_realtime_quotes(stk_code)
    current_price_map[stk_code] = float(df.iloc[0]["price"])

In [46]:
average = 0.0
for stk_code in watch_list:
    difference = 100 * (current_price_map[stk_code]/last_price_map[stk_code] - 1)
    average += difference
    if (difference > 3.0 or
        difference < -3.0):
        print (stk_code, difference)
average = average / len(watch_list)
print ("average is: ", average)

603859 3.04168231318
300334 3.13152400835
300649 5.71963394343
300675 4.54217736121
300552 3.34175793316
603233 3.43691148776
002900 4.09707724426
603822 3.31230283912
average is:  0.878492586255
